# [YouTube Data API v3 reference](https://developers.google.com/youtube/v3/docs)
***
# Imports

In [1]:
import os
from dotenv import load_dotenv
import requests
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import math
import psycopg2

In [2]:
# assume API calls are made at the same exact time

script_timestamp = datetime.now(timezone.utc) # YT API uses UTC timezone
script_timestamp

datetime.datetime(2024, 4, 29, 17, 42, 38, 248372, tzinfo=datetime.timezone.utc)

***
# Get API key and DB password

In [3]:
# Load environment variables from .env file
load_dotenv("environment_variables.env")

True

In [4]:
api_key = os.getenv("API_KEY")
psql_pw = os.getenv("PSQL_PW")

***
# Database Connection

In [5]:
host = "youtubeviewprediction.cd0c8oow2pnr.us-east-1.rds.amazonaws.com"
port = 5432
database = "postgres"
user = "postgres"

In [6]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=psql_pw
    )

    # Create a cursor object using the cursor() method
    cursor = connection.cursor()

    # Execute a SQL query
    cursor.execute("SELECT version();")

    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

You are connected to -  ('PostgreSQL 16.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',) 



***
# Most popular videos
## Extract

API limits 200 videos per call across 4 pages.

In [7]:
videos_api_url = "https://www.googleapis.com/youtube/v3/videos"

In [8]:
def parse_video_json(video):

    video_id = video["id"]

    # snippet
    publish_datetime = video["snippet"]["publishedAt"]
    channel_id = video["snippet"]["channelId"]
    video_title = video["snippet"]["title"]
    video_description = video["snippet"]["description"]
    channel_title = video["snippet"]["channelTitle"]
    num_tags = len(video["snippet"].get("tags", [])) # can be null
    category_id = video["snippet"]["categoryId"]

    # contentDetails
    duration = video["contentDetails"]["duration"]
    licensed_content = video["contentDetails"]["licensedContent"]

    # status
    made_for_kids = video["status"]["madeForKids"]

    # statistics
    num_views = video["statistics"]["viewCount"]
    num_likes = video["statistics"].get("likeCount", None) # can be null
    num_comments = video["statistics"].get("commentCount", None) # can be null

    return [video_id, 
            publish_datetime, 
            channel_id, 
            video_title,
            video_description,
            channel_title,
            num_tags,
            category_id, 
            duration, 
            licensed_content, 
            made_for_kids, 
            num_views, 
            num_likes, 
            num_comments]    

In [9]:
video_df = pd.DataFrame(columns=["script_timestamp", 
                                 "video_id", 
                                 "publish_datetime", 
                                 "channel_id",
                                 "video_title",
                                 "video_description",
                                 "channel_title",
                                 "num_tags", 
                                 "category_id",  
                                 "duration", 
                                 "licensed_content", 
                                 "made_for_kids", 
                                 "num_views", 
                                 "num_likes", 
                                 "num_comments"])
video_df

,script_timestamp,video_id,publish_datetime,channel_id,video_title,video_description,channel_title,num_tags,category_id,duration,licensed_content,made_for_kids,num_views,num_likes,num_comments


In [10]:
pageToken = ""

while pageToken is not None:
    params = {
        "key": api_key,
        "part": "id, snippet, contentDetails, status, statistics",
        "chart": "mostPopular",
        "hl": "en",
        "regionCode": "US",
        "maxResults": 50,
        "pageToken": pageToken
    }
    
    response = requests.get(videos_api_url, params=params)

    if response.status_code == 200:
        response_json = response.json()

        for video in response_json["items"]:
            # add video details and datetime of request to end of video dataframe
            video_df.loc[len(video_df)] = [script_timestamp] + parse_video_json(video)

        # get nextPageToken with null safety
        pageToken = response_json.get("nextPageToken", None)

    else:
        print(f"response.status_code = {response.status_code}")
        pageToken = None
    

In [11]:
video_df

,script_timestamp,video_id,publish_datetime,channel_id,video_title,video_description,channel_title,num_tags,category_id,duration,licensed_content,made_for_kids,num_views,num_likes,num_comments
0,2024-04-29 17:42:38.248372+00:00,cw_TI06Bfds,2024-04-28T19:37:00Z,UCwVg9btOceLQuNCdoQk9CXg,I Built a Theme Park In My House!,I built a GIANT theme park in my house and inv...,Ben Azelart,27,24,PT24M36S,True,False,4283340,66997,5131
1,2024-04-29 17:42:38.248372+00:00,78fQeeEcWMQ,2024-04-29T08:01:17Z,UCooVYzDxdwTtGYAkcPmOgOw,How to get a FREE HYPERCHARGE SKIN!,You WILL get a Hypercharge skin for FREE when ...,Brawl Stars,21,20,PT2M7S,True,False,7928850,198738,7961
2,2024-04-29 17:42:38.248372+00:00,DxHw4UdDJDY,2024-04-28T16:43:03Z,UCja7QUMRG9AD8X2F_vXFb9A,I Built a Secret Room in the MALL! Ft/ Ben Aze...,I Built a Secret Room in the MALL! Today Ferra...,The Royalty Family,31,22,PT31M20S,True,False,2677439,48792,5187
3,2024-04-29 17:42:38.248372+00:00,K_DSF87l8C8,2024-04-29T09:19:56Z,UCVg9nCmmfIyP4QcGOnZZ9Qg,Street Fighter 6 - Akuma Gameplay Trailer,Akuma will unleash his rage on foes worthy eno...,Street Fighter,4,20,PT3M4S,True,False,402557,34446,2872
4,2024-04-29 17:42:38.248372+00:00,q7H_qhIwmuc,2024-04-29T05:28:23Z,UCU7iRrk3xfpUk0R6VdyC1Ow,Inside the NBA Reacts To Timberwolves SWEEPING...,Watch highlights from Inside the NBA with Shaq...,NBA on TNT,22,17,PT8M56S,True,False,691422,11409,2082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2024-04-29 17:42:38.248372+00:00,erLbbextvlY,2024-03-30T16:00:01Z,UCX6OQ3DkcsbYNE6H8uQQuVA,7 Days Stranded On An Island,I can’t believe we actually did this\nSend mon...,MrBeast,0,24,PT22M26S,True,False,135127613,5088445,140963
196,2024-04-29 17:42:38.248372+00:00,PVsyDvu9sg4,2024-04-23T19:30:52Z,UCiGnwS3OVEuqbc78ELEFDeQ,we tried building a house in black & white in ...,this might be our first house fail??\nyou can ...,CarynandConnieGaming,28,20,PT28M30S,True,False,523242,39636,3312
197,2024-04-29 17:42:38.248372+00:00,Dm8OjeNNgqE,2024-04-23T15:15:02Z,UCXrExb1m3VjR0m1_l9luxRQ,Can MrSavage Beat PWR...,Today PWR is dropping into a game but against ...,PWR,28,20,PT17M59S,True,False,1039800,26047,1146
198,2024-04-29 17:42:38.248372+00:00,mPVjyFRjyro,2024-04-23T18:03:18Z,UC5UYMeKfZbFYnLHzoTJB1xA,Every Christopher Nolan Movie Ranked,Head to http://squarespace.com/schaffrillas to...,Schaffrillas Productions,15,24,PT59M29S,True,False,477692,26404,2607


In [12]:
video_df.isna().sum()

script_timestamp     0
video_id             0
publish_datetime     0
channel_id           0
video_title          0
video_description    0
channel_title        0
num_tags             0
category_id          0
duration             0
licensed_content     0
made_for_kids        0
num_views            0
num_likes            1
num_comments         1
dtype: int64

## Transform

In [13]:
# fix datatypes
video_df.dtypes

script_timestamp     datetime64[ns, UTC]
video_id                          object
publish_datetime                  object
channel_id                        object
video_title                       object
video_description                 object
channel_title                     object
num_tags                           int64
category_id                       object
duration                          object
licensed_content                    bool
made_for_kids                       bool
num_views                         object
num_likes                         object
num_comments                      object
dtype: object

In [14]:
video_df.sample(1)

,script_timestamp,video_id,publish_datetime,channel_id,video_title,video_description,channel_title,num_tags,category_id,duration,licensed_content,made_for_kids,num_views,num_likes,num_comments
119,2024-04-29 17:42:38.248372+00:00,uDzAYs329Ig,2024-04-25T22:00:39Z,UC5f5IV0Bf79YLp_p9nfInRA,How Many Lava Lamps Stop A Wrench Throw?,Buy Rexy Clothing ➤ https://rexyclothing.com\n...,How Ridiculous,3,17,PT20M40S,True,False,704911,29059,2306


### Turn publish_datetime into datetime

In [15]:
# turn publish_datetime into datetime
video_df["publish_datetime"] = pd.to_datetime(video_df["publish_datetime"], format="ISO8601")

### Parse duration in minutes from duration

In [16]:
video_df["duration"]

0      PT24M36S
1        PT2M7S
2      PT31M20S
3        PT3M4S
4       PT8M56S
         ...   
195    PT22M26S
196    PT28M30S
197    PT17M59S
198    PT59M29S
199    PT38M44S
Name: duration, Length: 200, dtype: object

In [17]:
video_df["duration"] = pd.to_timedelta(video_df["duration"]).apply(lambda x: x.seconds)
video_df.rename(columns={"duration": "duration_seconds"}, inplace=True)

### Turn num_view, num_likes, and num_comments into ints

In [18]:
video_df[["num_views", "num_likes", "num_comments"]].isna().sum()

num_views       0
num_likes       1
num_comments    1
dtype: int64

In [19]:
video_df[["num_views", "num_likes", "num_comments"]] = video_df[["num_views", "num_likes", "num_comments"]].fillna(0)
video_df[["num_views", "num_likes", "num_comments"]].isna().sum()

num_views       0
num_likes       0
num_comments    0
dtype: int64

In [20]:
video_df[["num_views", "num_likes", "num_comments"]] = video_df[["num_views", "num_likes", "num_comments"]].astype(dtype="int")

In [21]:
video_df.dtypes

script_timestamp     datetime64[ns, UTC]
video_id                          object
publish_datetime     datetime64[ns, UTC]
channel_id                        object
video_title                       object
video_description                 object
channel_title                     object
num_tags                           int64
category_id                       object
duration_seconds                   int64
licensed_content                    bool
made_for_kids                       bool
num_views                          int64
num_likes                          int64
num_comments                       int64
dtype: object

In [23]:
video_df["category_id"].apply(lambda x: len(x)).value_counts()

category_id
2    188
1     12
Name: count, dtype: int64

## Load

In [25]:
video_fact = """CREATE TABLE IF NOT EXISTS video_fact
(
 collected_at timestamp NOT NULL,
 video_id     varchar(11) NOT NULL,
 num_views    int NOT NULL,
 num_likes    int NOT NULL,
 num_comments int NOT NULL,
 CONSTRAINT PK_1 PRIMARY KEY ( collected_at, video_id )
);"""

cursor.execute(video_fact)

In [27]:
video_dim = """CREATE TABLE IF NOT EXISTS video_dim
(
 video_id      varchar(11) NOT NULL,
 channel_id    varchar(24) NOT NULL,
 title         varchar(100) NOT NULL,
 description   varchar(5000) NOT NULL,
 num_tags      int NOT NULL,
 duration_sec  int NOT NULL,
 licensed      boolean NOT NULL,
 made_for_kids boolean NOT NULL,
 published_at  timestamp NOT NULL,
 category_id   int NOT NULL,
 CONSTRAINT PK_2 PRIMARY KEY ( video_id )
);"""
cursor.execute(video_dim)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


***
# Channel metadata from top 200 videos
## Extract
API limits 50 channels per call with no pages.

In [63]:
unique_channel_ids = video_df["channel_id"].unique()

In [59]:
channels_api_url = "https://www.googleapis.com/youtube/v3/channels"

In [60]:
def parse_channel_json(channel):
    channel_id = channel["id"]

    # snippet
    channel_created_datetime = channel["snippet"]["publishedAt"]
    channel_name = channel["snippet"]["title"]

    # statistics
    channel_total_views = channel["statistics"].get("viewCount", None)
    channel_num_subscribers = channel["statistics"].get("subscriberCount", None)
    channel_num_videos = channel["statistics"].get("videoCount", None)

    return [channel_id, 
            channel_created_datetime,
            channel_name, 
            channel_total_views, 
            channel_num_subscribers,
            channel_num_videos]    

In [64]:
len(unique_channel_ids) % 50

33

In [65]:
# https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def chunks(lst, n):
    return [lst[i:i + n] for i in range(0, len(lst), n)]

channels_chunked = chunks(unique_channel_ids, 50)
[len(chunk) for chunk in channels_chunked] # chunks = # API calls

[50, 50, 50, 33]

In [66]:
channel_df = pd.DataFrame(columns=["request_datetime", 
                                "id", 
                                "created_datetime", 
                                "name"
                                "total_views", 
                                "num_subscribers",
                                "num_videos"])
channel_df

,request_datetime,id,created_datetime,total_views,num_subscribers,num_videos


In [67]:
for channels in channels_chunked:
    params = {
        "key": api_key,
        "part": "id, snippet, statistics",
        "id": ", ".join(channels),
        "maxResults": 50,
    }

    response = requests.get(channels_api_url, params=params)

    if response.status_code == 200:
        response_json = response.json()

        for channel in response_json["items"]:
            # add channel details and datetime of request to end of video dataframe
            channel_df.loc[len(channel_df)] = [script_timestamp] + parse_channel_json(channel)

    else:
        print(f"response.status_code = {response.status_code}")

In [68]:
channel_df.shape

(183, 6)

## Transform

In [69]:
# fix datatypes
channel_df.dtypes

request_datetime    datetime64[ns, UTC]
id                               object
created_datetime                 object
total_views                      object
num_subscribers                  object
num_videos                       object
dtype: object

### Turn created_datetime into datetime

In [70]:
channel_df["created_datetime"] = pd.to_datetime(channel_df["created_datetime"], format="ISO8601")

### Turn total_views, num_subscribers, num_videos into ints

In [71]:
channel_df[["total_views", "num_subscribers", "num_videos"]].isna().sum()

total_views        0
num_subscribers    0
num_videos         0
dtype: int64

In [72]:
channel_df[["total_views", "num_subscribers", "num_videos"]] = channel_df[["total_views", "num_subscribers", "num_videos"]].fillna(0)
channel_df[["total_views", "num_subscribers", "num_videos"]].isna().sum()

total_views        0
num_subscribers    0
num_videos         0
dtype: int64

In [73]:
channel_df[["total_views", "num_subscribers", "num_videos"]] = channel_df[["total_views", "num_subscribers", "num_videos"]].astype(dtype="int")

In [74]:
channel_df.dtypes

request_datetime    datetime64[ns, UTC]
id                               object
created_datetime    datetime64[ns, UTC]
total_views                       int64
num_subscribers                   int64
num_videos                        int64
dtype: object

## Load

In [ ]:
channel_dim = """CREATE TABLE IF NOT EXISTS channel_dim
(
 channel_id varchar(24) NOT NULL,
 name       varchar(36) NOT NULL,
 created_at timestamp NOT NULL,
 CONSTRAINT PK_1 PRIMARY KEY ( channel_id )
);"""
cursor.execute(channel_dim)

In [ ]:
channel_fact = """CREATE TABLE IF NOT EXISTS channel_fact
(
 datetime        timestamp NOT NULL,
 channel_id      varchar(24) NOT NULL,
 total_views     int NOT NULL,
 num_subscribers int NOT NULL,
 num_videos      int NOT NULL,
 CONSTRAINT PK_1 PRIMARY KEY ( datetime, channel_id )
);"""
cursor.execute(channel_fact)

# Video Categories

In [83]:
videoCategories_api_url = "https://www.googleapis.com/youtube/v3/videoCategories"

## Extract

In [80]:
def parse_videoCategories_json(category):
    id = category["id"]
    name = category["snippet"]["title"]

    return [id, name]

In [82]:
videoCategories_df = pd.DataFrame(columns=["id","name"])
videoCategories_df

,id,name


In [ ]:
params = {
    "key": api_key,
    "part": "snippet",
    "regionCode": "US",
}

response = requests.get(videoCategories_api_url, params=params)

if response.status_code == 200:
    response_json = response.json()

    for category in response_json["items"]:
        # add channel details and datetime of request to end of video dataframe
        videoCategories_df.loc[len(videoCategories_df)] = parse_videoCategories_json(category)

else:
    print(f"response.status_code = {response.status_code}")

In [84]:
videoCategories_df

,id,name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [87]:
# all categories accounted for
set(videoCategories_df["id"]).intersection(set(video_df["category_id"])) == set(video_df["category_id"])

True

In [89]:
videoCategories_df["name"].apply(lambda x: len(x)).max()

21

## Load

In [ ]:
categories_dim = """CREATE TABLE IF NOT EXISTS categories_dim
(
 category_id int NOT NULL,
 name        varchar(21) NOT NULL,
 CONSTRAINT PK_1 PRIMARY KEY ( category_id )
);"""
cursor.execute(categories_dim)